In [1]:
#from _future_ import print_function

from pyspark.sql import SparkSession
# $example on:schema_merging$
from pyspark.sql import Row
# $example off:schema_merging$

from pyspark.sql.window import Window

from pyspark.sql.functions import dense_rank

In [2]:
ss = SparkSession \
    .builder \
    .appName("Python Spark SQL data source example") \
    .getOrCreate()
path = "D://_E/Spark_training/SparkPythonTraining/Task4/"  

In [3]:
df_p = ss.read.load(path + "products.csv",
                         format="csv", sep=";", inferSchema="true", header="true")
#df_p.show()

In [4]:
df_oi = ss.read.load(path + "order_items.csv",
                         format="csv", sep=";", inferSchema="true", header="true")
#df_oi.show()

In [5]:
df_s = df_oi.join(df_p, df_oi.product_id == df_p.product_id, 'inner')\
    .select (df_p.category, df_p.name,(df_oi.quantity * df_p.price).alias('cost'))\
    .groupby( df_p.category, df_p.name)\
    .sum ('cost')\
    .withColumnRenamed('sum(cost)', 'cost')   
#df_s.show() 

In [6]:
w =  Window.partitionBy(df_s.category ).orderBy(df_s.cost.desc())

In [7]:
df_out = df_s.select( "category", "name", dense_rank().over(w).alias("rank"), "cost")\
    .filter("rank <= 3").select("category", "name","cost")\
    .sort(["category","cost"], ascending=[True,False])
#df_out.show()

In [9]:
df_out.write.csv(path + "answer.csv", sep=';')

In [10]:
df_out.repartition(1).write.csv(path + "cc_out.csv", sep=';', header = 'true')

In [11]:
ss.stop()